# Simulator

In [1]:
import sys
sys.path.insert(0, "..")
import pandas as pd
import matplotlib.pyplot as plt
from trading_simulator import TradingSimulator, DLSimulator
from dl_solutions.dlsolutions import CryptoDLSolutions
from data_processor.data_processing import DataProcessor

Using TensorFlow backend.


## Classic

In [ ]:
cryptos = ['ADA', 'BTC', 'ETH', 'LTC', 'LNK']
sel = 0

In [12]:
for crypto in cryptos:
    processor = DataProcessor([crypto])
    sim = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2, log = False)
    sim.simulate()

Loading... ADA


KeyboardInterrupt: 

In [ ]:
processor = DataProcessor([crypto])
sim1 = TradingSimulator(processor, crypto, strategy = [1], loss_allowed = 0.2)
sim2 = TradingSimulator(processor, crypto, strategy = [2], loss_allowed = 0.2)
sim3 = TradingSimulator(processor, crypto, strategy = [3], loss_allowed = 0.2)
sim4 = TradingSimulator(processor, crypto, strategy = [4], loss_allowed = 0.2)
sim5 = TradingSimulator(processor, crypto, strategy = [5], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [6], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [7], loss_allowed = 0.2)
sim7 = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2)

In [ ]:
sim1.simulate()

In [ ]:
sim2.simulate()

In [ ]:
sim3.simulate()

In [ ]:
sim4.simulate()

In [ ]:
sim5.simulate()

In [ ]:
sim6.simulate()

In [ ]:
sim7.simulate()

## DL Solution

In [7]:
crypto = 'ETH'
periods_to_retraining = 1000
prev_periods = 5
pred_periods = 10
model_selector = 'lstm'
columns = ['RSI', 'close']
#columns = ['weekday_num', 'RSI', 'MACD', 'close']
#columns = ['RSI', 'MACD', 'close']
#columns = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'RSI', 'MACD', 'close']
#columns = ['SO_K', 'SMA 50', 'SMA 200', 'close']
columns = ['close']
num_features = len(columns)
target = None
#target = 'close'

processor = DataProcessor([crypto])
sim = DLSimulator(processor, crypto, periods_to_retraining, prev_periods, pred_periods, model_selector, columns, target)
# TODO Add target if we want features as timestamps
#sim.simulate()
df = sim.get_df()
real_df = df.loc[327:1577]
real_df = df.loc[327:1577]
to_pred_df = df.loc[1578:1578]
real_and_pred = df.iloc[:1001]
real_df

# PROBAR GENERAR DATASET CON SALTOS DE X DIAS A CONOCER

Loading... ETH
Extracting columns columns for ETH
Proccessing and arranging columns for LSTM model


,close_4,close_3,close_2,close_1,close_0,close
327,467.87,470.47,485.57,486.77,471.71,468.57
328,470.47,485.57,486.77,471.71,432.48,449.04
329,485.57,486.77,471.71,432.48,444.99,460.23
330,486.77,471.71,432.48,444.99,431.14,457.47
331,471.71,432.48,444.99,431.14,432.46,450.05
...,...,...,...,...,...,...
1573,4511.21,4215.73,4117.25,4196.44,4347.59,3958.11
1574,4215.73,4117.25,4196.44,4347.59,4306.40,3875.42
1575,4117.25,4196.44,4347.59,4306.40,4436.91,3959.93
1576,4196.44,4347.59,4306.40,4436.91,4105.64,3924.14


In [8]:
norm_strat = 2
model_sel = 0
layers = 3
neurons = [50, 50, 50, 50]
batch_size = 64
epochs = 150
activations = ['relu', 'sigmoid']
losses = ['mse', 'binary_crossentropy']
activation = 'relu'
loss = 'mse'
metrics = ['accuracy']
optimizer = 'adam'
initial_learning_rate = 0.01

if target != None: 
    num_timestamps = None
else:
    num_timestamps = prev_periods

lstm = CryptoDLSolutions(
    real_df, 
    norm_strat, 
    model_sel, 
    layers, 
    neurons, 
    batch_size, 
    epochs, 
    num_timestamps, 
    num_features, 
    activation, 
    loss, 
    metrics, 
    optimizer, 
    initial_learning_rate)

lstm.build()
print(lstm.get_model().summary())
lstm.train()

Input shape: (1251, 5, 1) (1251, 1) (0, 5, 1) (0, 1)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 5, 50)             10400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 10)             2440      
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
dropout_5 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 10)                840       
_________________________________________________________________
d

In [12]:
pred = lstm.predict()
print('pred:', pred)

testx (1, 5, 1)
preds [[0.53046995]]
preds rev norm [[2589.81676615]]
real [[4014.15]]
Test RMSE: 1424.333
pred: [[2589.81676615]]


In [9]:
to_pred_df = df.loc[1578:1578]
to_pred_df

,close_4,close_3,close_2,close_1,close_0,close
1578,4306.4,4436.91,4105.64,3897.94,4089.37,4014.15


In [10]:
lstm.set_test(to_pred_df)

In [ ]:
df[-20:]

,close_1,MACD_1,RSI_1,weekday_num_1,close_0,MACD_0,RSI_0,weekday_num_0,close,MACD,RSI,weekday_num
1569,4582.95,36.473333,55.716077,2.0,4511.21,43.536946,53.835840,3,4130.00,-57.803905,46.707531,6.0
1570,4511.21,43.536946,53.835840,3.0,4215.73,25.003914,46.826530,4,3781.93,-87.109146,39.777252,0.0
1571,4215.73,25.003914,46.826530,4.0,4117.25,2.342833,44.735958,5,3858.99,-102.929159,41.834910,1.0
1572,4117.25,2.342833,44.735958,5.0,4196.44,-9.121124,46.793026,6,4019.08,-101.380062,45.965427,2.0
1573,4196.44,-9.121124,46.793026,6.0,4347.59,-5.941362,50.574650,0,3958.11,-103.874758,44.664534,3.0
1574,4347.59,-5.941362,50.574650,0.0,4306.40,-6.668199,49.541304,1,3875.42,-111.241886,42.891556,4.0
1575,4306.40,-6.668199,49.541304,1.0,4436.91,3.249386,52.829940,2,3959.93,-109.004601,45.282169,5.0
1576,4436.91,3.249386,52.829940,2.0,4105.64,-15.443552,44.841174,3,3924.14,-108.864567,44.433846,6.0
1577,4105.64,-15.443552,44.841174,3.0,3897.94,-46.481683,40.686916,4,3944.73,-105.871725,45.071392,0.0
1578,3897.94,-46.481683,40.686916,4.0,4089.37,-54.998832,45.681743,5,4014.15,-96.782613,47.268176,1.0


In [ ]:
to_pred_df = pd.read_csv('/Users/gonzalozabala/Documents/Master IA/AIMaster/TFM/Code/crypto_bot/Datasets/ETH.csv', header=[1])
to_pred_df = pd.DataFrame(to_pred_df['close'].iloc[:10]).T
to_pred_df['y'] = 686
to_pred_df

In [ ]:
processor = DataProcessor(['ETH'])
processor.load_data()
processor.clean_data('ETH')
#processor.feature_extraction('ETH')
#columns = ['close', 'Open Close Difference', 'Volume USDT' ,'Result']
#columns = ['close']
#df = processor.feature_selection('ETH', columns) 
df.loc[1588]